# 問題
問題42において、実験設定を変化させると正解率が変化するかどうかを調べよ。実験設定の例としては、大規模言語モデルの温度パラメータ、プロンプト、多肢選択肢の順番、多肢選択肢の記号などが考えられる。

正解の選択肢を全てDに入れ替えて解答させる例。

In [3]:
import pandas as pd
import cohere
import os
from dotenv import load_dotenv
# ---csvの読み込み---
path = "/Users/nakamuratuzumi/Jupyterlab/言語処理100本ノック 2025/第05章_大規模言語モデル/High School Chemistry.csv"
df = pd.read_csv(path,
                header=None,
                names= ["question", "a", "b", "c", "d", "answer"])
# --- リストに変換 ---
rows = []  # 空のリストを用意
for row in df.itertuples(index=False, name=None):
    rows.append(tuple(row))

import json, time

def build_batch_prompt(batch_rows):
    head = ('次の各問題に答えてください。出力は必ずJSON配列のみ：'
            '[{"index":1,"answer":"A|B|C|D"}, ...]。説明や余計な文は書かない。')
    lines = [head, ""]
    for i, (q,a,b,c,d,_gold) in enumerate(batch_rows, start=1):
        lines += [f"{i}. 問題: {q}",
                  f"  A: {a}",
                  f"  B: {b}",
                  f"  C: {c}",
                  f"  D: {d}", ""]
    return "\n".join(lines)

all_answers = []
batch_size = 10               # 1リクエスト=10問
max_rpm = 9                   # 1分あたり9回まで（安全マージン）
sleep_between = 60.0 / max_rpm

# 環境変数からAPIキーを読み込み
load_dotenv()

# Cohereクライアント初期化
co = cohere.Client(os.environ["COHERE_API_KEY"])

for i in range(0, len(rows), batch_size):
    batch = rows[i:i+batch_size]
    prompt = build_batch_prompt(batch)

    # 429対策：リトライ&スロットリング
    for attempt in range(5):
        try:
            resp = co.chat(model="command-r-plus-08-2024",
                           message=prompt,
                           temperature=0.8)
            break
        except cohere.error.TooManyRequestsError:
            # バーストしたら少し待って再試行（指数バックオフ）
            time.sleep(2 ** attempt)
    else:
        raise RuntimeError("429が解消しませんでした。")

    # JSONだけ返すよう指示している想定。壊れた場合のフォールバック付き
    try:
        arr = json.loads(resp.text)
        arr = sorted(arr, key=lambda x: x["index"])
        all_answers.extend([item["answer"] for item in arr])
    except Exception:
        # フォールバック：各行から A/B/C/D を拾う
        lines = [t.strip() for t in resp.text.splitlines() if t.strip()]
        for ln in lines:
            if ln[-1:] in {"A","B","C","D"}:
                all_answers.append(ln[-1:])
            # 足りない分はスキップせず埋めたいなら適宜処理

    # 次のコールまで小休止してRPMを守る
    time.sleep(sleep_between)

all_answers

['D',
 'A',
 'B',
 'A',
 'C',
 'B',
 'C',
 'B',
 'C',
 'A',
 'A',
 'A',
 'D',
 'C',
 'C',
 'D',
 'B',
 'D',
 'D',
 'A',
 'D',
 'B',
 'A',
 'C',
 'D',
 'C',
 'B',
 'B',
 'A',
 'D',
 'C',
 'A',
 'A',
 'C',
 'D',
 'A',
 'C',
 'C',
 'B',
 'D',
 'B',
 'C',
 'C',
 'B',
 'B',
 'D',
 'C',
 'B',
 'C',
 'C',
 'A',
 'D',
 'C',
 'B',
 'D',
 'C',
 'B',
 'C',
 'C',
 'B',
 'A',
 'D',
 'D',
 'B',
 'B',
 'A',
 'B',
 'D',
 'B',
 'B',
 'C',
 'B',
 'A',
 'C',
 'B',
 'B',
 'D',
 'C',
 'B',
 'C',
 'A',
 'B',
 'B',
 'C',
 'B',
 'D',
 'C',
 'B',
 'C',
 'D',
 'C',
 'A',
 'D',
 'B',
 'D',
 'B',
 'A',
 'A',
 'C',
 'B',
 'B',
 'C',
 'D',
 'B',
 'C',
 'D',
 'B',
 'A',
 'B',
 'C',
 'D',
 'C',
 'B',
 'B',
 'D',
 'A',
 'C',
 'B',
 'C',
 'B',
 'A',
 'B',
 'D',
 'B',
 'D',
 'B',
 'B',
 'A',
 'D',
 'D',
 'A|B|C',
 'D',
 'B',
 'B',
 'C',
 'C',
 'A',
 'D',
 'B',
 'B',
 'A',
 'C',
 'C',
 'C',
 'A',
 'B',
 'C',
 'B',
 'C']

In [4]:
point = 0
for answer, collect_answer in zip(all_answers, df["answer"]):
  if answer == collect_answer:
    point += 1

ratio = f"{round(point/len(all_answers) * 100, 2)}%"
ratio 

'52.35%'